<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [65]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [66]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [67]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [68]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

     |████████████████████████████████| 12.0MB 2.6MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-cp37-none-any.whl size=12019125 sha256=1e274207fd3610cf1bbabc1622003ea7a0f4c739e8c523d5e7f3c34b70dfe979
  Stored in directory: /Users/shannonhallmark/Library/Caches/pip/wheels/48/5c/1c/15f9d02afc8221a668d2172446dd8467b20cdb9aef80a172a4
Successfully built en-core-web-sm
Note: you may need to restart the kernel to use updated packages.


In [69]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import spacy

In [71]:
text = yelp['text'].values.tolist()

In [72]:
def tokenize(document):
    doc = nlp(document)
    return [token.lemma.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [73]:
vect = CountVectorizer(stop_words='english')

# tokenize and build vocab
vect.fit(text) # Fit then transform. 

# transform text
dtm = vect.transform(text)

# Create a Vocabulary

In [74]:
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())

In [75]:
# TFIDFVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
# We want to control the value of the length of documents. 
tfidf = TfidfVectorizer(stop_words='english', min_df=0.025, max_df=.98, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
# Similar to fit_predict
dtm = tfidf.fit_transform(text)

# Print word counts
# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,10,15,20,30,able,absolutely,actually,ago,amazing,area,...,wonderful,work,working,worst,worth,wouldn,wrong,year,years,yelp
0,0.0,0.000000,0.239091,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.123631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.146278,0.0,0.0
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.298333,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.225699,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [76]:
# Calculate Distance of TF-IDF Vectors
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix  = cosine_similarity(dtm)
df = pd.DataFrame(dist_matrix)

In [77]:
df.shape

(10000, 10000)

In [78]:
df[0].sort_values(ascending=False)[:10]

0       1.000000
6899    0.512933
3714    0.501494
2131    0.456473
5465    0.439754
2865    0.433492
8470    0.408057
8531    0.404986
9036    0.403812
8411    0.401263
Name: 0, dtype: float64

In [79]:
# Nearest Neighbors
# Instantiate
from sklearn.neighbors import NearestNeighbors

# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [80]:
nn.kneighbors([dtm.iloc[0].values])

(array([[0.        , 0.98698182, 0.9985046 , 1.        , 1.        ]]),
 array([[   0, 6899, 3714,  469, 3543]]))

In [81]:
random_review = [ """Chocolate was so good it gave the diabetes."""]

In [82]:
new = tfidf.transform(random_review)

In [83]:

nn.kneighbors(new.todense())

(array([[0.66237392, 0.74142786, 0.94871147, 0.99045888, 1.        ]]),
 array([[8765, 4654, 5450, 2575, 6311]]))

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [84]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [85]:
vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [86]:
pipe = Pipeline([('vect', vect), ('clf', rfc)])

In [87]:

parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

In [88]:
y = yelp['stars']
X = yelp['text']

In [89]:
# Train/Test Split
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=23)

In [90]:
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   29.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [91]:
pred = grid_search.predict(yelp['text'])

In [92]:
print("Total number of star predictions: ",len(pred))
print(grid_search.best_score_)
print(pred)

Total number of star predictions:  10000
0.5429850746268656
[1 5 4 ... 4 5 1]


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [93]:
!conda install gensim -y

done

# All requested packages already installed.



In [94]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [95]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [96]:
def tokenize2(text):
    tokens = []
    for i in range(len(text)):
        tokens.append([token for token in simple_preprocess(text[i]) if token not in STOPWORDS])
    return tokens;

In [97]:
tokens = tokenize2(text)

In [98]:
def tokenize1(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [99]:
tokenize1("This is my really good review on Chocolate-Mountain Chocolate store.")

['good', 'review', 'chocolate', 'mountain', 'chocolate', 'store']

In [100]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]
for t in topics: 
    print(t)

place food good time like
great food good service place
food good great service like
good place food like time
place food good service great
food service place time great
food place good great like
great place good time like
great good food place time
place good food great service


In [101]:
id2word = Dictionary(tokens)

In [102]:
id2word.doc2bow(tokenize1("This is my really good review on Chocolate-Mountain Chocolate store."))

[(40, 1), (599, 1), (812, 1), (1540, 2), (1556, 1)]

In [103]:
len(id2word.keys())
id2word.filter_extremes(no_below=5, no_above=0.95)

In [104]:
len(id2word.keys())

7493

Create a bag of words representation of the entire corpus

In [105]:
corpus = [id2word.doc2bow(text_bow) for text_bow in tokens]

Your LDA model should be ready for estimation: 

In [106]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [107]:
lda.print_topics()

[(0,
  '0.011*"service" + 0.009*"great" + 0.008*"good" + 0.008*"time" + 0.008*"food" + 0.007*"place" + 0.007*"like" + 0.005*"got" + 0.005*"love" + 0.004*"best"'),
 (1,
  '0.014*"place" + 0.011*"food" + 0.009*"good" + 0.009*"great" + 0.009*"service" + 0.008*"like" + 0.007*"time" + 0.006*"got" + 0.005*"ve" + 0.004*"best"'),
 (2,
  '0.013*"good" + 0.012*"place" + 0.011*"food" + 0.009*"like" + 0.008*"time" + 0.006*"great" + 0.005*"best" + 0.005*"service" + 0.005*"order" + 0.004*"nice"'),
 (3,
  '0.016*"great" + 0.013*"food" + 0.012*"place" + 0.010*"good" + 0.008*"service" + 0.008*"like" + 0.007*"time" + 0.005*"ve" + 0.005*"best" + 0.004*"definitely"'),
 (4,
  '0.011*"food" + 0.010*"great" + 0.010*"place" + 0.009*"service" + 0.008*"good" + 0.008*"time" + 0.006*"like" + 0.005*"got" + 0.005*"best" + 0.004*"nice"'),
 (5,
  '0.011*"food" + 0.010*"good" + 0.009*"place" + 0.008*"time" + 0.007*"like" + 0.006*"service" + 0.006*"great" + 0.006*"got" + 0.005*"ve" + 0.004*"ordered"'),
 (6,
  '0.014*"p

Learn the vocubalary of the yelp data:

In [108]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [109]:
topics = [' '.join(t[0:5]) for t in words]

In [110]:
!pip install pyLDAvis

In [111]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [112]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.005753 -0.000183       1        1  10.421571
9      0.003914 -0.002590       2        1  10.288714
2     -0.002811  0.001923       3        1  10.248651
1     -0.000221  0.000457       4        1  10.134301
0     -0.002231 -0.003655       5        1  10.092881
7      0.003095  0.004607       6        1   9.993627
8     -0.001164 -0.005093       7        1   9.868268
4     -0.000713  0.001026       8        1   9.723052
6     -0.002154  0.000952       9        1   9.714558
5     -0.003469  0.002555      10        1   9.514368, topic_info=    Category         Freq      Term        Total  loglift  logprob
196  Default  4268.000000     great  4268.000000  30.0000  30.0000
138  Default  4922.000000      food  4922.000000  29.0000  29.0000
127  Default  4727.000000     place  4727.000000  28.0000  28.0000
39   Default  4619.000000      good  4619.000000  27.0000  27.0000
44   Default  3631.000000   service  3631.000000  26.0000  26.0000
..       ...          ...       ...          ...      ...      ...
29   Topic10   114.672958      said  1204.412964   0.0007  -5.9149
116  Topic10   131.526535      best  1812.253662  -0.2708  -5.7777
125  Topic10   119.846428    people  1432.195312  -0.1284  -5.8707
45   Topic10   118.108620     staff  1561.373535  -0.2293  -5.8854
38   Topic10   112.470886  friendly  1385.150391  -0.1585  -5.9343

[851 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
181       1  0.087567  absolutely
181       2  0.139634  absolutely
181       3  0.113601  absolutely
181       4  0.097034  absolutely
181       5  0.087567  absolutely
...     ...       ...         ...
1588      6  0.125929          yo
1588      7  0.083952          yo
1588      8  0.083952          yo
1588      9  0.083952          yo
1588     10  0.083952          yo

[5050 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 10, 3, 2, 1, 8, 9, 5, 7, 6])

In [113]:
lda[corpus[0]]

[(0, 0.123652816),
 (1, 0.09971049),
 (2, 0.13406782),
 (3, 0.08646417),
 (4, 0.11467223),
 (5, 0.09028874),
 (6, 0.06629588),
 (7, 0.136151),
 (8, 0.0844351),
 (9, 0.06426181)]

In [114]:
distro = [lda[d] for d in corpus]

In [115]:
def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [116]:
d_dist = {k:0 for k in range(0,15)}

In [117]:
d_dist

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0}

In [118]:
for col in yelp.columns:
    print(col)

business_id
cool
date
funny
review_id
stars
text
useful
user_id


In [120]:
from gensim.models.coherencemodel import CoherenceModel

In [124]:
def coherence(dictionary, corpus, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word, workers=4)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    
    return model_list, coherence_values

Based on my results from the topic model, and the fact that the reviews were from retaurants, the overall topic seems to be a restaurant. It seems that the people that attended the restaurant enjoyed the food and atmosphere. I'm not 100% sure of the topic accuracy because i ran into some errors and i ran out of time.